In [1]:
!pip install -U tensorflow-addons

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
import random

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *

#You can import other libraries if you want

In [3]:
###### DO NOT MODIFY THIS PART
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed) # fix your random seed : In same model, you produce the same result
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1) 
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)
######

In [4]:
artists = pd.read_csv('./train/artists.csv')
# Sort artists by number of paintings
artists = artists.sort_values(by=['paintings'], ascending=False)

# Create a dataframe with artists having more than 200 paintings
artists_top = artists[artists['paintings'] >= 0].reset_index()
artists_top = artists_top[['name', 'paintings']]

updated_name = "Albrecht_Dürer".replace("_", " ")
artists_top.iloc[4, 0] = updated_name

In [5]:
# Explore images of top artists
images_dir = './train/images' #Type Your Own Data Directory
artists_dirs = os.listdir(images_dir)
artists_top_name = artists_top['name'].str.replace(' ', '_').values

# See if all directories exist
for name in artists_top_name:    
    if os.path.exists(os.path.join(images_dir, name)):  # images_idr: directory folder // name: label (y) 
        print("find -->", os.path.join(images_dir, name))
    else:
        print("Did not find -->", os.path.join(images_dir, name))

find --> ./train/images\Vincent_van_Gogh
find --> ./train/images\Edgar_Degas
find --> ./train/images\Pablo_Picasso
find --> ./train/images\Pierre-Auguste_Renoir
find --> ./train/images\Albrecht_Dürer
find --> ./train/images\Paul_Gauguin
find --> ./train/images\Francisco_Goya
find --> ./train/images\Rembrandt
find --> ./train/images\Alfred_Sisley
find --> ./train/images\Titian
find --> ./train/images\Marc_Chagall


In [6]:
batch_size = 16 # Type your own batch size
input_shape = (256, 256, ) # Type your own input shape
n_classes = artists_top.shape[0]

# Image data generate with validation data fraction of 0.2 and data rescale 1/255.
train_datagen = ImageDataGenerator(validation_split=0.2,
                                   rescale=1./255,
                                   rotation_range=90,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  )

# Generate train data
train_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    shuffle=True,
                                                    classes=artists_top_name.tolist()
                                                   )

# Generate validation data
valid_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    subset="validation",
                                                    shuffle=True,
                                                    classes=artists_top_name.tolist()
                                                   )

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
print("Total number of batches =", STEP_SIZE_TRAIN, "and", STEP_SIZE_VALID)

Found 3092 images belonging to 11 classes.
Found 767 images belonging to 11 classes.
Total number of batches = 193 and 47


In [7]:
#######################
# Make your own model #
#######################

# VGG16
# input_shape=(256, 256, 3)
model = Sequential()

# Convolutional layers
model.add(Conv2D(input_shape=(256, 256, 3), filters=64, kernel_size=(3, 3), padding='same', activation='relu'))

# Block 1
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu', name='Block1_Conv1'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu', name='Block1_Conv2'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='Block1_Pool'))

# Block 2
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu', name='Block2_Conv1'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu', name='Block2_Conv2'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='Block2_Pool'))

# Block 3
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', name='Block3_Conv1'))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', name='Block3_Conv2'))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', name='Block3_Conv3'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='Block3_Pool'))

# Block 4
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu', name='Block4_Conv1'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu', name='Block4_Conv2'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu', name='Block4_Conv3'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='Block4_Pool'))

# Block 5
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu', name='Block5_Conv1'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu', name='Block5_Conv2'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu', name='Block5_Conv3'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='Block5_Pool'))

# Fully Connected Layer
model.add(Flatten(name='Flatten'))
model.add(Dense(units=4096, activation='relu', name='FC1'))
model.add(Dropout(0.5, name='Dropout1'))
model.add(Dense(units=4096, activation='relu', name='FC2'))
model.add(Dropout(0.5, name='Dropout2'))
model.add(Dense(units=n_classes, activation='softmax', name='Predictions'))

# Model description
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 Block1_Conv1 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 Block1_Conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 Block1_Pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 Block2_Conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 Block2_Conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 Block2_Pool (MaxPooling2D)  (None, 64, 64, 128)       0

In [8]:
# Training

# Compile model
# Adam optimizer, loss function and metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

n_epoch = 300

history = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                              epochs=n_epoch)

c:\Users\genie\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/300


: 

: 

In [ ]:
his = {}
his['loss'] = history.history['loss']
his['accuracy'] = history.history['accuracy'] 
his['val_loss'] = history.history['val_loss']
his['val_accuracy'] = history.history['val_accuracy']

# Plot the training graph
def plot_training(history):
    acc = history['accuracy']
    val_acc = history['val_accuracy']
    loss = history['loss']
    val_loss = history['val_loss']
    epochs = range(len(acc))

    fig, axes = plt.subplots(1, 2, figsize=(15,5))
    
    axes[0].plot(epochs, acc, 'r-', label='Training Accuracy')
    axes[0].plot(epochs, val_acc, 'b--', label='Validation Accuracy')
    axes[0].set_title('Training and Validation Accuracy')
    axes[0].legend(loc='best')

    axes[1].plot(epochs, loss, 'r-', label='Training Loss')
    axes[1].plot(epochs, val_loss, 'b--', label='Validation Loss')
    axes[1].set_title('Training and Validation Loss')
    axes[1].legend(loc='best')
    
    plt.show()
    
plot_training(his)

In [ ]:
model.save("./model")